In [1]:
%matplotlib inline
import torch
from sebm.supervised_clf import init_clf
dataset =  'mnist' # 'svhn' # 'cifar10' # 'mnist' #  'flowers102' #
if dataset == 'mnist' or dataset =='fashionmnist':
    input_channels, im_height, im_width = 1, 28, 28
else:
    input_channels, im_height, im_width = 3, 32, 32
device = torch.device('cuda:0')
arch =  'simplenet' # 'mlp' # 
seed = 1
lr = 1e-4
heldon = -1
channels, kernels, strides, paddings =[64,64,32,32], [3,4,4,4], [1,2,2,2], [1,1,1,1]
hidden_dim, latent_dim = [128], 10
activation = 'ReLU'
data_dir = '../../../sebm_data/'
load_version = 'clf-out=%s-d=%s-seed=%d-lr=%s-arch=%s' % (heldon, dataset, seed, lr, arch)

if arch == 'simplenet':
    clf = init_clf(arch=arch,
                    im_height=im_height, 
                    im_width=im_width, 
                    input_channels=input_channels, 
                    channels=channels, 
                    kernels=kernels, 
                    strides=strides, 
                    paddings=paddings, 
                    hidden_dim=hidden_dim,
                    latent_dim=latent_dim,
                    activation=activation)
else:
    raise NotImplementError

clf = clf.cuda().to(device)
print('Loading trained weights..')
clf.load_state_dict(torch.load('../weights/cp-%s' % load_version)['clf_state_dict'])

ImportError: cannot import name 'init_clf' from 'sebm.supervised_clf' (/home/hao/Research/sebm/sebm/supervised_clf.py)

In [11]:
torch.Tensor([0,0, 1,0,0]).argmax()

tensor(2)

In [8]:
from sebm.eval import Evaluator_CLF
evaluator = Evaluator_CLF(clf, device, dataset, data_dir)
evaluator.test_accuracy()

Computing the test accuracy..
Using downloaded and verified file: ../../../sebm_data/SVHN/test_32x32.mat
Dataset=svhn, Test Accuracy=0.9228


In [11]:
y.shape

(60000,)

In [31]:
from collections import defaultdict
dataset = 'cifar10'
d = defaultdict(list)
for shot in [1, 10, 100, -1]:
    for seed in range(1, 6, 1):
        fin = open('../results/log-semi_clf-data=%s-shots=%d-seed=%d-lr=0.0001.txt' % (dataset, shot, seed), 'r')
        for line in fin.readlines():
            line = line.strip().split(', ')
            if line[0] == 'Epoch=100':
#                 print(float(line[-1][6:]))
                d[shot].append(float(line[-1][7:]))
        fin.close

In [33]:
d

defaultdict(list,
            {1: [0.1953, 0.1451, 0.2054, 0.1086, 0.1673],
             10: [0.2722, 0.2036, 0.2602, 0.2225, 0.2365],
             100: [0.3674, 0.398, 0.3962, 0.3955, 0.3968],
             -1: [0.7033, 0.713, 0.7032, 0.7087, 0.7251]})

In [32]:
import numpy as np
for key in d.keys():
    print('shot=%d, accu=%.4f', (key, np.array(d[key]).mean()))

shot=%d, accu=%.4f (1, 0.16434)
shot=%d, accu=%.4f (10, 0.23900000000000002)
shot=%d, accu=%.4f (100, 0.39078)
shot=%d, accu=%.4f (-1, 0.7106600000000001)


In [13]:
shot, seed, dataset = 1, 1, 'mnist'
fin = open('../results/log-semi_clf-data=%s-shots=%d-seed=%d-lr=0.0001.txt' % (dataset, shot, seed), 'r')
fin.seek(2, 2)
line = fin.readline()
line = line.strip()
print(line)
fin.close

UnsupportedOperation: can't do nonzero end-relative seeks